### The Following setup is taken from the Mask_RCNN repo by matterport and Deep Learning with Python by Chollet. 

We import our packages, including maskrcnn, which needs to be installed from the github repo. 

We also set up our directories and paths before we organize our data into tensors. 

In the future, we may want to look into the official Keras implementation of Mask_RCNN, which is currently in development. This implementation is pretty well developed though and uses tensorflow/keras as backend.

In [2]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt


# Import Mask RCNN
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn.model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "models")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

TRAIN_DIR = os.path.join(ROOT_DIR, 'data/raw/train')
VALIDATION_DIR = os.path.join(ROOT_DIR, 'data/raw/validation')
TEST_DIR = os.path.join(ROOT_DIR, 'data/raw/test')
try:
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    os.mkdir(test_dir)
except:
    FileExistsError

/opt/anaconda3/envs/maskrcnn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [62]:
next(os.walk(IMAGERY_DIR))

('/tana-crunch/deepimagery/data/raw/stephtest-subset/Imagery',
 [],
 ['02_R7_C7_OS_ms.tif',
  '02_R2_C0_GS_ms.tif',
  '03_R3_C4_OS_ms.tif',
  '02_R5_C3_OS_pan.tif',
  '03_R6_C3_GS_ms.tif',
  '08_R1_C0_OS_circles.png',
  '04_R2_C4_GS_pan.tif',
  '06_R4_C0_OS_pan.tif',
  '03_R1_C7_OS_ms.tif',
  '03_R4_C0_GS_ms.tif',
  '05_R7_C4_GS_pan.tif',
  '08_R1_C1_GS_pan.tif',
  '02_R5_C4_OS_ms.tif',
  '02_R0_C3_GS_ms.tif',
  '07_R1_C0_OS_pan.tif',
  '01_R6_C7_GS_pan.tif',
  '08_R7_C2_GS_circles.png',
  '03_R0_C3_OS_pan.tif',
  '05_R1_C0_GS_pan.tif',
  '06_R1_C7_OS_morph2.png',
  '06_R5_C6_GS_morph2.png',
  '06_R1_C3_OS_morph1.png',
  '06_R5_C2_GS_morph1.png',
  '08_R3_C6_GS_circles.png',
  '08_R0_C3_OS_morph1.png',
  '08_R4_C2_GS_morph1.png',
  '08_R0_C7_OS_morph2.png',
  '07_R7_C4_OS_pan.tif',
  '08_R4_C6_GS_morph2.png',
  '08_R7_C5_GS_pan.tif',
  '03_R6_C0_OS_morph1.png',
  '03_R2_C1_GS_morph1.png',
  '02_R3_C1_OS_ms.tif',
  '03_R6_C4_OS_morph2.png',
  '03_R2_C5_GS_morph2.png',
  '01_R0_C3_GS_pan

In [5]:
### Should our tensor be rank 4 [h, w, channels on and off season, instance] 
### or rank 5 [h, w, channels on season, instances, channels off season]
def load_merge_wv2(image_id):
    """Load the specified wv2 os/gs image pairs and return a [H,W,8] 
    Numpy array. Channels are ordered [B, G, R, NIR, B, G, R, NIR], OS 
    first.
    """
    # Load image
    os_path = IMAGERY_DIR+'/'+image_id+'_OS_ms.tif'
    gs_path = IMAGERY_DIR+'/'+image_id+'_GS_ms.tif'
    os_image = skimage.io.imread(os_path)
    gs_image = skimage.io.imread(gs_path)
    # If has more than 4 bands, select correct bands 
    # will need to provide image config in future
    # to programmaticaly use correct band mappings
    if os_image.shape[-1] != 4:
        os_image = np.dstack((os_image[:,:,1:3],os_image[:,:,4],os_image[:,:,6]))
    if gs_image.shape[-1] != 4:
        gs_image = np.dstack((gs_image[:,:,1:3],gs_image[:,:,4],gs_image[:,:,6]))
    stacked_image = np.dstack((os_image, gs_image))
    stacked_image_path = TRAIN_DIR +'/'+ image_id + '_OSGS_ms.tif'
    return (stacked_image_path, stacked_image)

IMAGERY_DIR = os.path.join(ROOT_DIR, 'data/raw/stephtest-subset/Imagery')
GROUNDTRUTH_DIR = os.path.join(ROOT_DIR, 'data/raw/stephtest-subset/Groundtruth')

# all files, including ones we don't care about
file_ids_all = next(os.walk(IMAGERY_DIR))[2]
# all multispectral on and off season tifs
image_ids_all = [image_id for image_id in file_ids_all if 'ms' in image_id]
#check for duplicates
print(len(image_ids_all) != len(set(image_ids_all)))

image_ids_gs = [image_id for image_id in image_ids_all if 'GS' in image_id]
image_ids_os = [image_id for image_id in image_ids_all if 'OS' in image_id]

#check for equality
print(len(image_ids_os) == len(image_ids_gs))

image_ids_short = [image_id[0:8] for image_id in image_ids_gs]
image_ids_short

stacked_dict = {}

for imid in image_ids_short:
    
    path, arr = load_merge_wv2(imid)
    stacked_dict.update({path:arr})
    
# trying to save 8 channel numpy array with GS, OS info. this is what matterport expects
# BUT in mold_inputs() in https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/model.py
# indicates that the input png/array must only have three channels...
# We could change mold_inputs to not have this requirement and also change
#         input_image = KL.Input(
#            shape=[None, None, 3], name="input_image")
# on line 1841 of mrcnn/model.py but not sure if this is all the changes
# that would be required
# this issue indicates fix is simpler: https://github.com/matterport/Mask_RCNN/issues/314

for key, val in stacked_dict.items():
    skimage.io.imsave(key,val,plugin='tifffile')

False
True


In [54]:
image_ids_os

['02_R7_C7_OS_ms.tif',
 '03_R3_C4_OS_ms.tif',
 '03_R1_C7_OS_ms.tif',
 '02_R5_C4_OS_ms.tif',
 '02_R3_C1_OS_ms.tif',
 '03_R7_C2_OS_ms.tif',
 '03_R5_C1_OS_ms.tif',
 '02_R1_C2_OS_ms.tif',
 '03_R2_C5_OS_ms.tif',
 '02_R6_C6_OS_ms.tif',
 '02_R4_C5_OS_ms.tif',
 '03_R0_C6_OS_ms.tif',
 '03_R6_C3_OS_ms.tif',
 '02_R2_C0_OS_ms.tif',
 '02_R0_C3_OS_ms.tif',
 '03_R4_C0_OS_ms.tif',
 '03_R1_C6_OS_ms.tif',
 '02_R5_C5_OS_ms.tif',
 '02_R7_C6_OS_ms.tif',
 '03_R3_C5_OS_ms.tif',
 '03_R5_C0_OS_ms.tif',
 '02_R1_C3_OS_ms.tif',
 '02_R3_C0_OS_ms.tif',
 '03_R7_C3_OS_ms.tif',
 '02_R4_C4_OS_ms.tif',
 '03_R0_C7_OS_ms.tif',
 '03_R2_C4_OS_ms.tif',
 '02_R6_C7_OS_ms.tif',
 '02_R0_C2_OS_ms.tif',
 '03_R4_C1_OS_ms.tif',
 '03_R6_C2_OS_ms.tif',
 '02_R2_C1_OS_ms.tif',
 '02_R1_C0_OS_ms.tif',
 '03_R5_C3_OS_ms.tif',
 '03_R7_C0_OS_ms.tif',
 '02_R3_C3_OS_ms.tif',
 '02_R5_C6_OS_ms.tif',
 '03_R1_C5_OS_ms.tif',
 '03_R3_C6_OS_ms.tif',
 '02_R7_C5_OS_ms.tif',
 '03_R4_C2_OS_ms.tif',
 '02_R0_C1_OS_ms.tif',
 '02_R2_C2_OS_ms.tif',
 '03_R6_C1_

In [3]:
class ImageryConfig(Config):
    """Configuration for training on worldview-2 imagery. 
    Will eventually want to make this a sub-class of a 
    larger Imagery class. Overrides values specific to WV2.
    
    Descriptive documentation for each attribute is at
    https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/config.py"""
    
    def __init__(self, N):
        """Set values of computed attributes. Channel dimension is overriden, 
        replaced 3 with N as per this guideline: https://github.com/matterport/Mask_RCNN/issues/314
        THERE MAY BE OTHER CODE CHANGES TO ACCOUNT FOR 3 vs N channels. See other 
        comments."""
        
        # Effective batch size
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT

        # Input image size
        if self.IMAGE_RESIZE_MODE == "crop":
            self.IMAGE_SHAPE = np.array([self.IMAGE_MIN_DIM, self.IMAGE_MIN_DIM, N])
        else:
            self.IMAGE_SHAPE = np.array([self.IMAGE_MAX_DIM, self.IMAGE_MAX_DIM, N])

        # Image meta data length
        # See compose_image_meta() for details
        self.IMAGE_META_SIZE = 1 + 3 + 3 + 4 + 1 + self.NUM_CLASSES

    
    # Give the configuration a recognizable name
    NAME = "wv2-subsets"

    # Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 2
    IMAGES_PER_GPU = 4

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + ag

    # Use small images for faster training. Determines the image shape.
    # From build() in model.py
    # Exception("Image size must be dividable by 2 at least 6 times "
                       #     "to avoid fractions when downscaling and upscaling."
                       #    "For example, use 256, 320, 384, 448, 512, ... etc. "
    IMAGE_MIN_DIM = 300
    IMAGE_MAX_DIM = 300

    # Use smaller anchors because our image and objects are small.
    # Setting Large upper scale since some fields take up nearly 
    # whole image
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 300)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 1000

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50

In [ ]:
class ImageryDataset(utils.Dataset):
    """Generates the Imagery dataset."""
    
    def load_single_image(self, image_id):
        """Load the specified image and return a [H,W,4] Numpy array.
        Channels are ordered [B, G, R, NIR].
        """
        # Load image
        image = skimage.io.imread(self.image_info[image_id]['path'])
        # If has more than 4 bands, select correct bands 
        # will need to provide image config in future
        # to programmaticaly use correct band mappings
        if image.shape[-1] != 4:
            image = np.dstack((image[:,:,1:3],image[:,:,4],image[:,:,6]))
        return image
    
    def load_wv2(self, dataset_dir, subset):
        """Loads the specified number of image metadata into
        the dataset from the training or val directory. Images should be 
        seperated into test and train beforehand (write function for this)
        Currently unclear to me if I need to explicitly create a Rank 4
        Tensor or if the maskrcnn train does that for me and I just need
        to supply the "dataset" which would only contain this metadata.
        
        The nucleus matterport example is most similar to ours, see their 
        github."""
        
        # add are only other class besides background
        self.add_class("wv2", 1, "agriculture")
        
        # add images to training dataset
        for i in range(count):
            self.add_image("wv2", image_id=i, 
                           path=os.path.join(TRAIN_DIR, imgname),
                          width=width, heigh=height, bg)
        
    
    def load_mask(self, image_id):
        """Load instance masks for given image ID.
        """
        info = self.image_info[image_id]
        shapes = info['shapes']
        count = len(shapes)
        mask = np.zeros([info['height'], info['width'], count], dtype=np.uint8)
        for i, (shape, _, dims) in enumerate(info['shapes']):
            mask[:, :, i:i+1] = self.draw_shape(mask[:, :, i:i+1].copy(),
                                                shape, dims, 1)
        # Handle occlusions
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(count-2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        # Map class names to class IDs.
        class_ids = np.array([self.class_names.index(s[0]) for s in shapes])
        return mask.astype(np.bool), class_ids.astype(np.int32)

In [4]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
dataset_train = utils.Dataset()
dataset_train.add_image()
dataset_train.load_image()
# dataset_train.prepare()